In [ ]:
#@title Install Dependencies

!pip install sentence-transformers
!pip install torch

In [ ]:
#@title Load Embedding Model from Huggingface
from sentence_transformers import SentenceTransformer, util

model_name = "Snowflake/snowflake-arctic-embed-m"    # @param {type:"string"}

model = SentenceTransformer(model_name)

In [ ]:
#@title Upload Intents and Examples "output.csv"
from google.colab import files
import csv

uploaded = files.upload()

for filename in uploaded.keys():
  print(f"Uploaded: {filename}")

def read_csv(filename):
  intents = []
  examples = []

  with open(filename, "r", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      intents.append(row["intent"])
      examples.append(row["example"])

  return intents, examples

intent_values, example_values = read_csv("output.csv")
print(example_values)

In [11]:
#@title Generate Embeddings from Examples in output.csv

examples_embeddings = model.encode(example_values, convert_to_tensor=True)

In [17]:
#@title Determine Intent from Query

query = "send human" # @param {type:"string"}

top_k = min(5, len(examples_embeddings))

query_embedding = model.encode(query, convert_to_tensor=True)

hits = util.semantic_search(query_embedding, examples_embeddings, top_k=5)
hits = hits[0]

print("Query:", query)
print("Top 5 similar examples:")

for hit in hits:
    print(intent_values[hit['corpus_id']], example_values[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))


Query: send human
Top 5 similar examples:
intent_feedback_any send feedback (Score: 0.9340)
intent_core_here Chat (Score: 0.9246)
intent_core_here Let's do it here (Score: 0.9238)
intent_core_here This chat (Score: 0.9204)
intent_closing_start end chat (Score: 0.9196)


In [16]:
#@title Compare from rasa data
from google.colab import files
import csv

test_uploaded = files.upload()

def read_csv(filename):
  intents = []
  examples = []

  with open(filename, "r", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      intents.append(row["intent"])
      examples.append(row["example"])

  return intents, examples


for filename in test_uploaded.keys():
  test_intent_values, test_example_values = read_csv(filename)

  success_count = 0
  for i in range(len(test_intent_values)):
    top_k = min(5, len(examples_embeddings))
    query_embedding = model.encode(test_example_values[i], convert_to_tensor=True)

    hits = util.semantic_search(query_embedding, examples_embeddings, top_k=5)
    hits = hits[0]

    if intent_values[hits[0]['corpus_id']] == test_intent_values[i]:
      print(f"Success {test_intent_values[i]}:  {test_example_values[i]}")
      success_count += 1
    else:
      print(f"Error. Expected: {test_intent_values[i]} but found {intent_values[hits[0]['corpus_id']]}: {test_example_values[i]}")

  print(f"Total: {success_count}/{len(test_intent_values)}")



Saving test-nlu.csv to test-nlu (3).csv
Success intent_data_privacy_policy:  Are people watching my conversation?
Success intent_data_privacy_policy:  How is my data being used?
Success intent_data_privacy_policy:  How are you ensuring my privacy?
Success intent_live_chat:  Do you support live chat?
Success intent_live_chat:  I want to talk to a live agent
Success intent_live_chat:  Give me a damn human being!!
Success intent_user_preferences:  Change my [email address]{"entity": "preference"}?
Success intent_user_preferences:  Where can I reset my [password]{"entity": "preference"}?
Success intent_user_preferences:  I want to change my [login]{"entity": "preference"}.
Success intent_user_preferences:  I want to change my [personal information]{"entity": "preference"}.
Success intent_out_of_scope:  Who started Red Hat?
Success intent_out_of_scope:  What products does Red Hat make?
Success intent_are_you_ai:  What are you using to run this?
Success intent_are_you_ai:  Are you using arti